In [ ]:
import json
import random
from datetime import datetime

# File paths for data persistence
EXAM_DB = "exams.json"
STUDENT_DB = "students.json"

# Question class
class Question:
    def __init__(self, question_id, text, options, correct_answer):
        self.question_id = question_id
        self.text = text
        self.options = options
        self.correct_answer = correct_answer

    def check_answer(self, answer):
        return answer == self.correct_answer

    def to_dict(self):
        return {
            "question_id": self.question_id,
            "text": self.text,
            "options": self.options,
            "correct_answer": self.correct_answer
        }

# Exam class
class Exam:
    def __init__(self, exam_id, name, questions=None):
        self.exam_id = exam_id
        self.name = name
        self.questions = questions if questions else []

    def add_question(self, question):
        self.questions.append(question)

    def get_random_questions(self, num_questions=5):
        return random.sample(self.questions, min(num_questions, len(self.questions)))

    def to_dict(self):
        return {
            "exam_id": self.exam_id,
            "name": self.name,
            "questions": [q.to_dict() for q in self.questions]
        }

# Result class
class Result:
    def __init__(self, student_name, exam_name, answers, score, total_questions):
        self.student_name = student_name
        self.exam_name = exam_name
        self.answers = answers
        self.score = score
        self.total_questions = total_questions
        self.timestamp = str(datetime.now())

    def to_dict(self):
        return {
            "student_name": self.student_name,
            "exam_name": self.exam_name,
            "answers": self.answers,
            "score": self.score,
            "total_questions": self.total_questions,
            "timestamp": self.timestamp
        }

# User Authentication
class User:
    def __init__(self):
        self.students = []
        self.admins = []
        self.load_users()

    def load_users(self):
        try:
            with open(STUDENT_DB, 'r') as f:
                data = json.load(f)
                self.students = data.get('students', [])
                self.admins = data.get('admins', [])
        except FileNotFoundError:
            self.students = []
            self.admins = []

    def save_users(self):
        with open(STUDENT_DB, 'w') as f:
            json.dump({"students": self.students, "admins": self.admins}, f)

    def add_admin(self, username, password):
        self.admins.append({"username": username, "password": password})
        self.save_users()

    def add_student(self, username, password):
        self.students.append({"username": username, "password": password})
        self.save_users()

    def authenticate(self, username, password):
        for user in self.students + self.admins:
            if user["username"] == username and user["password"] == password:
                return True
        return False

# Exam System
class ExamSystem:
    def __init__(self):
        self.exams = []
        self.users = User()
        self.load_exams()

    def load_exams(self):
        try:
            with open(EXAM_DB, 'r') as f:
                data = json.load(f)
                self.exams = [Exam(**exam_data) for exam_data in data]
        except FileNotFoundError:
            self.exams = []

    def save_exams(self):
        with open(EXAM_DB, 'w') as f:
            json.dump([exam.to_dict() for exam in self.exams], f)

    def create_exam(self, name):
        exam_id = len(self.exams) + 1
        exam = Exam(exam_id, name)
        self.exams.append(exam)
        self.save_exams()

    def create_question(self, exam_id, text, options, correct_answer):
        exam = next((exam for exam in self.exams if exam.exam_id == exam_id), None)
        if exam:
            question_id = len(exam.questions) + 1
            question = Question(question_id, text, options, correct_answer)
            exam.add_question(question)
            self.save_exams()

    def take_exam(self, username, exam_id, answers):
        exam = next((exam for exam in self.exams if exam.exam_id == exam_id), None)
        if exam:
            correct_answers = 0
            for i, ans in enumerate(answers):
                if exam.questions[i].check_answer(ans):
                    correct_answers += 1
            score = (correct_answers / len(exam.questions)) * 100
            result = Result(username, exam.name, answers, score, len(exam.questions))
            self.save_result(result)
            return result

    def save_result(self, result):
        result_data = result.to_dict()
        result_filename = f"result_{result.student_name}_{result.exam_name}.json"
        with open(result_filename, 'w') as f:
            json.dump(result_data, f)

    def view_results(self, username):
        result_files = [f"result_{username}_{exam.name}.json" for exam in self.exams]
        results = []
        for result_file in result_files:
            try:
                with open(result_file, 'r') as f:
                    result_data = json.load(f)
                    results.append(result_data)
            except FileNotFoundError:
                continue
        return results

    def list_exams(self):
        return [exam.name for exam in self.exams]

    def view_exam(self, exam_id):
        exam = next((exam for exam in self.exams if exam.exam_id == exam_id), None)
        if exam:
            return [{"question": q.text, "options": q.options} for q in exam.questions]
        return []

# Main Interface
def main():
    system = ExamSystem()

    # Admin login (hardcoded for simplicity)
    username = input("Enter username: ")
    password = input("Enter password: ")

    if system.users.authenticate(username, password):
        print(f"Welcome {username}!")
        if username in [admin["username"] for admin in system.users.admins]:
            print("Admin privileges granted.")
            while True:
                print("\n1. Create Exam\n2. Create Question\n3. List Exams\n4. Exit")
                choice = input("Enter choice: ")
                if choice == "1":
                    exam_name = input("Enter exam name: ")
                    system.create_exam(exam_name)
                elif choice == "2":
                    exam_id = int(input("Enter exam ID: "))
                    question_text = input("Enter question text: ")
                    options = input("Enter options (comma-separated): ").split(",")
                    correct_answer = input("Enter correct answer: ")
                    system.create_question(exam_id, question_text, options, correct_answer)
                elif choice == "3":
                    exams = system.list_exams()
                    for exam in exams:
                        print(f"- {exam}")
                elif choice == "4":
                    break
        else:
            print("Student privileges granted.")
            exams = system.list_exams()
            for exam in exams:
                print(f"Exam: {exam}")
            exam_name = input("Enter the exam you want to take: ")
            exam = next((exam for exam in system.exams if exam.name == exam_name), None)
            if exam:
                random_questions = exam.get_random_questions()
                print("\nExam Questions:")
                answers = []
                for q in random_questions:
                    print(f"{q.text}")
                    for i, option in enumerate(q.options):
                        print(f"{i+1}. {option}")
                    answer = input("Your answer: ")
                    answers.append(answer)
                result = system.take_exam(username, exam.exam_id, answers)
                print(f"Your Score: {result.score}%")
    else:
        print("Invalid credentials!")

if __name__ == "__main__":
    main()
